In [38]:
import numpy as np
import tensorflow as tf
import os
import cv2
import matplotlib.pyplot as plt
import PIL.Image as img
from tensorflow.keras.preprocessing import image
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten, ZeroPadding2D
from tensorflow.keras.layers import Conv2D, MaxPooling2D, AveragePooling2D, BatchNormalization, Add, Input
from keras.callbacks import ReduceLROnPlateau
from keras.utils import to_categorical

In [39]:
DATADIR_TRAIN = "C:/Users/Gurpreet/Documents/BME571 - ML in BME/COVID_CNN/Covid19-dataset/train"
DATADIR_TEST = "C:/Users/Gurpreet/Documents/BME571 - ML in BME/COVID_CNN/Covid19-dataset/test"
IMG_SIZE=224

In [40]:
X_train = []
for folder in os.listdir(DATADIR_TRAIN):
    sub_path = DATADIR_TRAIN + '/' + folder
    for image in os.listdir(sub_path):
        img_path = sub_path + '/' + image
        img = cv2.imread(img_path)
        img = cv2.resize(img,(224,224)) #VGG19 needs an input of shape 224x224
        X_train.append(img)

In [41]:
X_test = []
for folder in os.listdir(DATADIR_TEST):
    sub_path = DATADIR_TEST + '/' + folder
    for image in os.listdir(sub_path):
        img_path = sub_path + '/' + image
        img = cv2.imread(img_path)
        img = cv2.resize(img,(224,224)) #VGG19 needs an input of shape 224x224
        X_test.append(img)

In [42]:
train_data = ImageDataGenerator(rescale = 1/255.0,
                                validation_split=0.2)
test_data = ImageDataGenerator(rescale = 1/255.0,
                               horizontal_flip=True)

In [43]:
train_set = train_data.flow_from_directory(
                    DATADIR_TRAIN,
                    subset = 'training',
                    batch_size=32,
                    shuffle=True,
                    target_size=(IMG_SIZE,IMG_SIZE),
                    class_mode = 'sparse'
)
val_set = train_data.flow_from_directory(
                    DATADIR_TRAIN,
                    subset = 'validation',
                    batch_size=32,
                    shuffle=True,
                    target_size=(IMG_SIZE,IMG_SIZE),
                    class_mode = 'sparse'
)
test_set = test_data.flow_from_directory(
                    DATADIR_TEST,
                    batch_size=32,
                    shuffle=True,
                    target_size=(IMG_SIZE,IMG_SIZE),
                    class_mode = 'sparse'
)

Found 201 images belonging to 3 classes.
Found 50 images belonging to 3 classes.
Found 66 images belonging to 3 classes.


In [84]:
# input activation has the same dimension as the output activation.
def identity_block(X, kernel_size, filters):
    k = kernel_size
    # save the value to add later
    x_skip = X
    
    # Layer 1
    X = Conv2D(filters, kernel_size = (k, k), strides = (1,1), padding = 'same')(X)
    X = BatchNormalization(axis = 3)(X)
    X = Activation('relu')(X)

    # Layer 2
    X = Conv2D(filters, kernel_size = (k,k), strides = (1,1), padding = 'same')(X)
    X = BatchNormalization(axis = 3)(X)

    # Add skip value to main path
    X = Add()([X, x_skip])
    X = Activation('relu')(X)
    
    return X

In [85]:
# input and output dimensions don’t match up. 
# The difference with the identity block is that there is a CONV2D layer in the shortcut path.
def convolution_block(X, filters, s = 2):
    # save the value to add later
    x_skip = X
    
    # Layer 1
    X = Conv2D(filters, (3,3), padding='same', strides = (s, s))(X)
    X = BatchNormalization(axis=3)(X)
    X = Activation('relu')(X)
    
    # Layer 2
    X = Conv2D(filters, (3,3), padding = 'same')(X)
    X = BatchNormalization(axis=3)(X)
    
    # Processing skip with conv(1,1)
    x_skip = Conv2D(filters, (1,1), strides = (s,s))(x_skip)
    x_skip = BatchNormalization(axis=3)(x_skip)
    
    # Add skip to main path
    X = Add()([X, x_skip])     
    X = Activation('relu')(X)
    
    return X

In [86]:
def ResNet34(shape = (IMG_SIZE, IMG_SIZE, 3), classes = 3):
    X_input = Input(shape)
    # skipped max padding for now 
    # X = ZeroPadding2D((3, 3))(X_input)
    # initialize layer weight may be useful
    # STAGE 1: convolution w/ kernal size 7x7,  64 filters, with stride 2, followed by max pool
    X = Conv2D(64, kernel_size = (7,7), strides = 2, padding = 'same')(X_input)
    X = BatchNormalization(axis=3)(X)
    X = Activation('relu')(X)
    X = MaxPooling2D((3,3), strides=(2,2))(X)
    
    # STAGE 2
    X = convolution_block(X, filters = 64)
    X = identity_block(X, 3, 64)
    X = identity_block(X, 3, 64)

    # STAGE 3
    X = convolution_block(X, filters = 128)
    X = identity_block(X, 3, 128)
    X = identity_block(X, 3, 128)
    X = identity_block(X, 3, 128)
    
    # STAGE 4
    X = convolution_block(X, filters = 256)
    X = identity_block(X, 3, 256)
    X = identity_block(X, 3, 256)
    X = identity_block(X, 3, 256)
    X = identity_block(X, 3, 256)
    X = identity_block(X, 3, 256)
    
    # STAGE 5
    X = convolution_block(X, filters = 512)
    X = identity_block(X, 3, 512)
    X = identity_block(X, 3, 512)
    
    X = AveragePooling2D(2,2, padding='same')(X)
    
    # output layer
    X = Flatten()(X)
    X = Dense(classes, activation='softmax')(X)
    
    
    # Create model
    model = Model(inputs = X_input, outputs = X, name='ResNet34')
    return model

In [87]:
model = ResNet34(shape = (IMG_SIZE,IMG_SIZE, 3), classes = 3)

In [88]:
model.compile(
    optimizer='adam', # optimizer
    loss='sparse_categorical_crossentropy', # loss function to optimize 
    metrics=['accuracy'] # metrics to monitor
)

In [89]:
model.summary()

Model: "ResNet34"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_12 (InputLayer)          [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv2d_275 (Conv2D)            (None, 112, 112, 64  9472        ['input_12[0][0]']               
                                )                                                                 
                                                                                                  
 batch_normalization_243 (Batch  (None, 112, 112, 64  256        ['conv2d_275[0][0]']             
 Normalization)                 )                                                          

 conv2d_285 (Conv2D)            (None, 14, 14, 128)  8320        ['activation_269[0][0]']         
                                                                                                  
 add_119 (Add)                  (None, 14, 14, 128)  0           ['batch_normalization_251[0][0]',
                                                                  'conv2d_285[0][0]']             
                                                                                                  
 activation_271 (Activation)    (None, 14, 14, 128)  0           ['add_119[0][0]']                
                                                                                                  
 conv2d_286 (Conv2D)            (None, 14, 14, 128)  147584      ['activation_271[0][0]']         
                                                                                                  
 batch_normalization_252 (Batch  (None, 14, 14, 128)  512        ['conv2d_286[0][0]']             
 Normaliza

                                                                                                  
 conv2d_296 (Conv2D)            (None, 7, 7, 256)    590080      ['activation_280[0][0]']         
                                                                                                  
 batch_normalization_261 (Batch  (None, 7, 7, 256)   1024        ['conv2d_296[0][0]']             
 Normalization)                                                                                   
                                                                                                  
 add_124 (Add)                  (None, 7, 7, 256)    0           ['batch_normalization_261[0][0]',
                                                                  'activation_279[0][0]']         
                                                                                                  
 activation_281 (Activation)    (None, 7, 7, 256)    0           ['add_124[0][0]']                
          

                                                                                                  
 activation_290 (Activation)    (None, 4, 4, 512)    0           ['batch_normalization_270[0][0]']
                                                                                                  
 conv2d_306 (Conv2D)            (None, 4, 4, 512)    2359808     ['activation_290[0][0]']         
                                                                                                  
 batch_normalization_271 (Batch  (None, 4, 4, 512)   2048        ['conv2d_306[0][0]']             
 Normalization)                                                                                   
                                                                                                  
 conv2d_307 (Conv2D)            (None, 4, 4, 512)    131584      ['activation_289[0][0]']         
                                                                                                  
 add_129 (

In [90]:
model.fit(
    train_set,
    epochs = 8,
    batch_size = 32,
    validation_data=val_set)

Epoch 1/8
7/7 [==============================] - 8s 843ms/step - loss: 6.2215 - accuracy: 0.4876 - val_loss: 25883444.0000 - val_accuracy: 0.2800
Epoch 2/8
7/7 [==============================] - 5s 645ms/step - loss: 0.8692 - accuracy: 0.7065 - val_loss: 5992844.0000 - val_accuracy: 0.2800
Epoch 3/8
7/7 [==============================] - 5s 713ms/step - loss: 0.6517 - accuracy: 0.7612 - val_loss: 722982.5625 - val_accuracy: 0.2800
Epoch 4/8
7/7 [==============================] - 5s 644ms/step - loss: 0.5054 - accuracy: 0.8010 - val_loss: 68215.9609 - val_accuracy: 0.2800
Epoch 5/8
7/7 [==============================] - 5s 674ms/step - loss: 0.4353 - accuracy: 0.8159 - val_loss: 9605.8154 - val_accuracy: 0.2800
Epoch 6/8
7/7 [==============================] - 5s 650ms/step - loss: 0.4091 - accuracy: 0.8458 - val_loss: 4066.4290 - val_accuracy: 0.2800
Epoch 7/8
7/7 [==============================] - 5s 694ms/step - loss: 0.3511 - accuracy: 0.8756 - val_loss: 568.7180 - val_accuracy: 0.30

In [51]:
pred = model.predict(test_set)
final_pred = np.argmax(pred, axis =1)

3/3 [==============================] - 1s 153ms/step


In [52]:
final_pred

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
      dtype=int64)